In [97]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import pickle

In [2]:
data = pd.read_csv('데이터/petition_data_all_2019_01_09.csv')

In [3]:
data.head()

,num,category,start-days,end-days,person,progress,title,count,petition_overview
0,21,안전/환경,2017-08-19,2017-11-17,kakao - ***,청원종료,스텔라 데이지호에 대한 제안입니다.,9,\n\t\t\t\t\t\t\t\t\t\t\t스텔라 데이지호에 대한 제안입니다. \n...
1,22,기타,2017-08-19,2017-11-17,kakao - ***,청원종료,비리제보처를 만들어주세요.,17,\n\t\t\t\t\t\t\t\t\t\t\t현 정부에 국민들이 가장 원하는 것은 부...
2,23,미래,2017-08-19,2017-09-03,kakao - ***,청원종료,제2의 개성공단,0,\n\t\t\t\t\t\t\t\t\t\t\t만일 하시는 대통령님 및 각 부처 장관님...
3,24,일자리,2017-08-19,2017-08-26,kakao - ***,청원종료,공공기관 무조건적인 정규직전환을 반대합니다.,53,\n\t\t\t\t\t\t\t\t\t\t\t현정부에서 정규직 일자리를 늘리는 것에 ...
4,25,미래,2017-08-19,2017-09-03,kakao - ***,청원종료,제2의 개성공단,0,\n\t\t\t\t\t\t\t\t\t\t\t만일 하시는 대통령님 및 각 부처 장관님...


In [13]:
print(data.isnull().values.any())

False


In [15]:
len(data)

371230

In [46]:
data = data.drop_duplicates(subset='petition_overview', keep='last')

In [47]:
len(data)

352988

In [49]:
print(data.groupby('category').size().sort_values(ascending=False))

category
정치개혁           53258
기타             43175
인권/성평등         31114
안전/환경          27542
교통/건축/국토       25899
외교/통일/국방       24462
육아/교육          23557
보건복지           22330
일자리            20631
행정             18145
문화/예술/체육/언론    16571
미래             16114
경제민주화          15203
성장동력            6469
반려동물            3629
저출산/고령화대책       3199
농산어촌            1690
dtype: int64


In [43]:
category_selected = ['정치개혁', '인권/성평등', '안전/환경', '교통/건축/국토', '외교/통일/국방']

In [88]:
df = data[data['category'].isin(category_selected)]
df.head()

,num,category,start-days,end-days,person,progress,title,count,petition_overview
0,21,안전/환경,2017-08-19,2017-11-17,kakao - ***,청원종료,스텔라 데이지호에 대한 제안입니다.,9,\n\t\t\t\t\t\t\t\t\t\t\t스텔라 데이지호에 대한 제안입니다. \n...
6,27,정치개혁,2017-08-19,2017-09-18,naver - ***,청원종료,김이수 헌재소장 임명 재고 건의,1,\n\t\t\t\t\t\t\t\t\t\t\t문 대통령님께 묻습니다 (2017. 8....
12,33,인권/성평등,2017-08-19,2017-11-17,naver - ***,청원종료,납득할 수 있는 장애 재판정을 받고 싶습니다.,15,\n\t\t\t\t\t\t\t\t\t\t\t안녕하십니까? 저는 경직형 양마비 뇌병변...
14,35,외교/통일/국방,2017-08-19,2017-08-26,kakao - ***,청원종료,문재인정부는 신한국형 페리프로세스가 작성했습니다.,0,\n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님 신한국형 페리프로세스 로...
15,36,인권/성평등,2017-08-19,2017-08-26,naver - ***,청원종료,한국채식인구 100만명. 학교 급식 및 군대에서 현미채식 선택권을 보장해주십시오!,1,\n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님과 각 정부 인사분들께 마...


In [75]:
X = df['petition_overview']

In [76]:
y = df['category']

In [81]:
X[:10]

0     \n\t\t\t\t\t\t\t\t\t\t\t스텔라 데이지호에 대한 제안입니다. \n...
6     \n\t\t\t\t\t\t\t\t\t\t\t문 대통령님께 묻습니다 (2017. 8....
12    \n\t\t\t\t\t\t\t\t\t\t\t안녕하십니까? 저는 경직형 양마비 뇌병변...
14    \n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님 신한국형 페리프로세스 로...
15    \n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님과 각 정부 인사분들께 마...
17    \n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님 한중관계 사드 갈등 해소...
19    \n\t\t\t\t\t\t\t\t\t\t\t지금부터 67년전 1950년 6.25전쟁...
25    \n\t\t\t\t\t\t\t\t\t\t\t지금부터 67년전 1950년 6.25전쟁...
30    \n\t\t\t\t\t\t\t\t\t\t\t간절히 간절히 간청 드립니다. \n\n\...
54    \n\t\t\t\t\t\t\t\t\t\t\t※지장물관련 국토부의 해석 : 기존 건축...
Name: petition_overview, dtype: object

In [87]:
X = X.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [90]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

129820
32455
129820
32455


In [91]:
print(X_train.head())
print(y_train.head())

333598    봄에 이어 이번 가을 겨울에도 어김없이 미세먼지가 왔습니다 적어도 봄에는 마스크 안...
344228    부동산 계약 전산화 해야 합니다  부동산 계약 전산화 해야 합니다  부동산 계약 전...
300621    어제 뉴스를 보고 너무 어이가 없어서 올려봅니다 동료들한테 처음 들었을 때 너무 황...
285042    최근 국방의 의무에 관련하여 지속적으로 이야기가 나오고있습니다 국방의 의무에대하여 ...
26544                                          조두순 재심 해주세요 
Name: petition_overview, dtype: object
333598       안전/환경
344228    교통/건축/국토
300621      인권/성평등
285042    외교/통일/국방
26544        안전/환경
Name: category, dtype: object


In [95]:
okt = Okt()

C:\Users\user\Miniconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [99]:
X_train_morphs = []
for sentence in tqdm_notebook(X_train):
    temp_X = []
    temp_X = okt.morphs(sentence)
    X_train_morphs.append(temp_X)

In [100]:
X_test_morphs = []
for sentence in tqdm_notebook(X_test):
    temp_X = []
    temp_X = okt.morphs(sentence)
    X_test_morphs.append(temp_X)

In [101]:
with open('X_train_morphs.pickle', 'wb') as file:
    pickle.dump(X_train_morphs, file, pickle.HIGHEST_PROTOCOL)

In [102]:
with open('X_test_morphs.pickle', 'wb') as file:
    pickle.dump(X_test_morphs, file, pickle.HIGHEST_PROTOCOL)